# TES4Py #

## The Elder Scrolls IV - Oblivion Parser ##

In [140]:
!pip install boltons

You are using pip version 6.0.8, however version 6.1.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
  Running setup.py install for boltons


# On Break #

In [1]:
from mmap import mmap
from pathlib import Path
import struct
import functools
from collections import namedtuple
from enum import IntEnum
from boltons.dictutils import MultiDict
import weakref
import itertools
import numba
import collections
import PyQt5

In [4]:
class Flags:
    def __init__(self, val, fields=None, **kwargs):
        self._flags = fields or kwargs
        self._val = val
        
    def __getattr__(self, flag):
        try:
            return bool(self._flags[flag] & self._val)
        except KeyError as ex:
            raise AttributeError from ex
            
    def __repr__(self):
        return '<Flags ' + ', '.join(
            '{!s}={!r}'.format(flag, getattr(self, flag))
            for flag in self._flags
        ) + '>'     

In [5]:
class StructFieldMeta(type):
    def __getitem__(self, offset):
        return self()[offset]
    
class StructField(metaclass=StructFieldMeta):
    def __init__(self):
        self.offset = None
    
    def transform(self, buffer):
        raise NotImplemented
    
    def __get__(self, instance, cls):
        return self.transform(instance.buffer[self.offset])
        
    def __getitem__(self, offset):
        self.offset = offset
        return self
    
class FixStrField(StructField):
    def transform(self, buffer):
        return buffer.tobytes().decode('latin1')

class ULongField(StructField):
    def __init__(self, int_type=None):
        self._int_type = int_type
        #assert issubclass(self._int_type, int)
        
    def transform(self, buffer):
        val = int.from_bytes(buffer, 'little', signed=False)
        if self._int_type:
            val = self._int_type(val)
        return val
    
class NamedTupleField(StructField):
    def __init__(self, struct_fmt, name, fields):
        self._struct_fmt = struct_fmt
        self._namedtuple = namedtuple(name, fields)
    
    def transform(self, buffer):
        return self._namedtuple(*struct.unpack_from(self._struct_fmt, buffer))
    
class FlagsField(StructField):
    def __init__(self, **flags):
        self._flags = flags
    
    def transform(self, buffer):
        val = int.from_bytes(buffer, 'little', signed=False)
        return Flags(val, self._flags)

In [18]:
class EspEsmFormat(collections.abc.Mapping):
    def __init__(self, path):
        self.path = path if isinstance(path, Path) else Path(str(path))
        self._num_groups = None
    
    def __enter__(self):
        self.file = f = self.path.open("r+b")
        self.mmap = mm = mmap(f.fileno(), 0)
        self.view = memoryview(mm)
        return self
    
    def __exit__(self, *args, **kwargs):
        del self.view
        self.mmap.close()
        del self.mmap
        self.file.close()
    
    @property
    def header(self):
        return Record(self.view)
    
    @property
    def groups(self):
        groups = {}
        bytes_consumed = self.header.total_size
        while bytes_consumed < len(self.view):
            group = Group(self.view[bytes_consumed:])
            yield group
            bytes_consumed += group.total_size
            
    def __iter__(self):
        for group in self.groups:
            yield group.label
    
    def __len__(self):
        if self._num_groups is None:
            ng = 0
            for group in self.groups:
                ng += 1
            self._num_groups = ng
        return self._num_groups
    
    def __getitem__(self, key):
        for group in self.groups:
            if group.label == key:
                return group
        raise KeyError(key)

In [7]:
class GroupType(IntEnum):
    top=0
    world_children=1
    interior_cell_block=2
    interior_cell_subblock=3
    exterior_cell_block=4
    exterior_cell_subblock=5
    cell_children=6
    topic_children=7
    cell_persistent=8,
    cell_temporary_children=9
    cell_visible_distant_children=10
    
class Group:
    def __init__(self, buffer):
        self._buffer = buffer
        assert self.type == 'GRUP'
    
    @property
    def buffer(self):
        return self._buffer
    
    @property
    def record_buffer(self):
        return self.buffer[self.header_size:]
    
    header_size = 20
    
    type = FixStrField[0:4]
    _size = ULongField[4:8]
    label = FixStrField[8:12]
    group_type = ULongField(GroupType)[12:16]
    stamp = ULongField[16:20]
    
    @property
    def total_size(self):
        return self._size
    
    @property
    def size(self):
        return self._size - self.header_size
    
    @property
    def records(self):
        bytes_consumed = 0
        while bytes_consumed < self.size:
            record = Record(self.record_buffer[bytes_consumed:])
            bytes_consumed += record.total_size
            yield record

In [8]:
class Record(collections.abc.Mapping):
    def __init__(self, buffer):
        self._buffer = buffer
        
        #self._subrecord_cache = MultiDict()
        #for subrecord in self.subrecords:
        #    self._subrecord_cache[subrecord.type] = subrecord
        self._num_subrecords = None
    
    @property
    def buffer(self):
        return self._buffer
    
    header_size = 20
    
    type = FixStrField[0:4]
    size = ULongField[4:8]
    flags = FlagsField(
        isesm=0x01,
        deleted=0x20,
        cast_shadows=0x200,
        is_quest_item=0x400,
        is_persistent=0x400,  # means "is quest item" or "is persistent" depending on context            
        initially_disabled=0x800,
        ignored=0x1000,
        visible_when_distant=0x8000,
        dangerous_off_limits=0x20000,
        is_compressed=0x40000,
        cant_wait=0x80000,
    )[8:12]
    
    formid = ULongField[12:16]
    vc_info = NamedTupleField('<BBH', 'VCInfo', ['day', 'month', 'owner'])[16:20]
    
    @property
    def subrecord_buffer(self):
        return self.buffer[self.header_size:self.header_size + self.size]
    
    @property
    def subrecords(self):
        bytes_consumed = 0
        while bytes_consumed < self.size:
            subrecord = SubRecord(self.subrecord_buffer[bytes_consumed:])
            bytes_consumed += subrecord.total_size
            yield subrecord
    
    def __iter__(self):
        for subrecords in self.subrecords:
            yield subrecords.type
    
    @property
    def total_size(self):
        return self.size + self.header_size
    
    def __len__(self):
        if self._num_subrecords is None:
            nr = 0
            for sr in self:
                nr += 1
            self._num_subrecords = nr
        return self._num_subrecords
        
    def __getitem__(self, key): 
        for subrecord in self.subrecords:
            if subrecord.type == key:
                return subrecord
        raise KeyError(key)

In [29]:
class SubRecord:
    def __init__(self, buffer):
        self._buffer = buffer
        #self._parent = weakref.ref(parent)
    
    @property
    def buffer(self):
        return self._buffer
    
    @property
    def total_size(self):
        return self.size + self.header_size
    
    header_size = 6
    type = FixStrField[0:4]
    size = ULongField[4:6]
    
    @property
    def zstring(self):        
        return self.buffer[self.header_size:self.total_size - 1].tobytes().decode('latin1')
        #return b[:b.find(b'\0')].decode('latin1')
        
    formid = ULongField[header_size:header_size+4]
    item_data = NamedTupleField('<Lf', 'ItemData', ['gold_value', 'weight'])[header_size:header_size+8]
    

In [33]:
esm = EspEsmFormat('E:/HDSteamLib/steamapps/common/Oblivion/data/Oblivion.esm')
esm.__enter__()
clot_r = next(esm['CLOT'].records)
clot_r['DATA'].item_data, clot_r['FULL'].zstring, list(clot_r.keys())

(ItemData(gold_value=8, weight=4.0),
 "Ciirta's Robes",
 ['EDID',
  'FULL',
  'ENAM',
  'BMDT',
  'MODL',
  'MODB',
  'MOD2',
  'MO2B',
  'ICON',
  'DATA'])

In [32]:
l = None
def getlistofrecs():
    global l
    l = [getattr(clot.get('EDID'), 'zstring', None) for clot in esm['ARMO'].records]
%timeit getlistofrecs()

10 loops, best of 3: 46.7 ms per loop


In [23]:
l

['SE32CirionsHelmet4',
 'SE32CirionsHelmet3',
 'SE32CirionsHelmet2',
 'SE32CirionsHelmet1',
 'SE32CirionsHelmet6',
 'SE32CirionsHelmet5',
 'SEMadnessMagicShield2',
 'SEMadnessMagicShield1',
 'SEMadnessMagicHelmet2',
 'SEMadnessMagicHelmet1',
 'SEMadnessMagicGreaves2',
 'SEMadnessMagicGreaves1',
 'SEMadnessMagicGauntlets2',
 'SEMadnessMagicGauntlets1',
 'SEMadnessMagicCuirass2',
 'SEMadnessMagicCuirass1',
 'SEMadnessMagicBoots2',
 'SEMadnessMagicBoots1',
 'SEAmberMagicShield2',
 'SEAmberMagicShield1',
 'SEAmberMagicHelmet2',
 'SEAmberMagicHelmet1',
 'SEAmberMagicGreaves2',
 'SEAmberMagicGreaves1',
 'SEAmberMagicGauntlets2',
 'SEAmberMagicGauntlets1',
 'SEAmberMagicCuirass2',
 'SEAmberMagicCuirass1',
 'SEAmberMagicBoots2',
 'SEAmberMagicBoots1',
 'SEMadnessShield2',
 'SEMadnessHelmet2',
 'SEMadnessGreaves2',
 'SEMadnessGauntlets2',
 'SEMadnessCuirass2',
 'SEMadnessBoots2',
 'SEMadnessShield1',
 'SEMadnessHelmet1',
 'SEMadnessGreaves1',
 'SEMadnessGauntlets1',
 'SEMadnessCuirass1',
 'SEMa